In [0]:
from google.colab import drive
drive.mount('/content/drive')

**tài liệu tham khảo**
+ https://forum.machinelearningcoban.com/t/vnlp-core-3-bai-toan-phan-loai-van-ban-phan-tich-cam-xuc-cua-binh-luan-text-classification/2371
+ https://forum.machinelearningcoban.com/t/vnlp-core-1-bai-toan-tach-tu-tieng-viet-tokenization-word-segmentation/2002
+ https://forum.machinelearningcoban.com/t/vnlp-core-2-thuc-hanh-training-va-su-dung-bieu-dien-tu-trong-khong-gian-word-embedding/2101?fbclid=IwAR2n3avtbLPavpxl9Mb6yBiV1xgR-udAKOIjRoo9v2tonILNFgkfZ6-84yk

+ f1_score : https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9

In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

In [0]:
import os
import pprint
import tensorflow as tf

if 'COLAB_TPU_ADDR' not in os.environ:
  print('ERROR: Not connected to a TPU runtime; please see the first cell in this notebook for instructions!')
else:
  tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
  print ('TPU address is', tpu_address)

  with tf.Session(tpu_address) as session:
    devices = session.list_devices()
    
  print('TPU devices:')
  pprint.pprint(devices)

In [0]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import random
from scipy.sparse import csr_matrix
from scipy.sparse import coo_matrix, hstack, vstack

from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout
from keras.constraints import max_norm


import math
from sklearn.model_selection import train_test_split

from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers
from sklearn.metrics import classification_report,confusion_matrix

In [0]:
from keras.optimizers import SGD

In [0]:
# x_train = pd.ExcelFile('/content/drive/My Drive/Colab Notebooks/project_final/quora-insincere-questions-classification/quora_train.xlsx')
# quora_train = x_train.parse('Sheet1')
# x_test = pd.ExcelFile('/content/drive/My Drive/Colab Notebooks/project_final/quora-insincere-questions-classification/quora_val.xlsx')
# quora_test = x_test.parse('Sheet1')

x_train = pd.ExcelFile('/content/drive/My Drive/quora-insincere-questions-classification/quora_train.xlsx')
quora_train = x_train.parse('Sheet1')
x_test = pd.ExcelFile('/content/drive/My Drive/quora-insincere-questions-classification/quora_val.xlsx')
quora_test = x_test.parse('Sheet1')

In [0]:
print(quora_train.shape)
print(quora_test.shape)
quora_train.head(2)

In [0]:
quora_test.head(2)

In [0]:
quora_train = quora_train.replace(np.nan, '')
quora_test = quora_test.replace(np.nan, '')

In [0]:
sentence_train = quora_train.text_norm1.values
sentence_test = quora_test['text_norm1'].values

In [0]:
y_train = quora_train.target.values
y_test = quora_test.target.values

In [0]:
def count_words(statement):
  statement = statement.replace('  ', ' ')
  return len(statement.split())

In [0]:
quora_train['num_words_2'] = quora_train.text_norm1.apply(count_words)

In [0]:
quora_train.num_words_2.value_counts()

In [0]:
quora_train.num_words_2.describe()

In [0]:
# str_sentence_train = " ".join(sentence_train).replace('  ', ' ')
# ls_word = list(set(str_sentence_train.split()))
# len(ls_word)

# tokenizer the sentences

In [0]:
embeddings_index = {}

# f = open('/content/drive/My Drive/Colab Notebooks/project_final/quora-insincere-questions-classification/embeddings/glove.840B.300d/glove.840B.300d.txt')
f = open('/content/drive/My Drive/quora-insincere-questions-classification/embeddings/glove.840B.300d/glove.840B.300d.txt')

In [0]:
for line in tqdm(f):
    values = line.split(" ")
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [0]:
len(embeddings_index)

# create embedding matrix

In [0]:
def the_norm_text(text, maxlen = 30):       
  # add empty string into text
  ls_seq =  text.split() + ['0']*( maxlen - len(text.split()))
  return " ".join(ls_seq)

In [0]:
def text_to_vector(text):
  '''
  input:(str) text
  output:(list) array( 1, 300*30) have 1 vecter size (, 300*30)
          represent a text is a vector to have 
  '''
  
  # in this problem
  # vì phần lớn text_norm1 có dộ dài nhỏ hơn 30, 1 vài câu là độ dài lớn hơn 30
  # choose maxlen = 30 , because we remove excess character
  # if senquence have not enough 30 words
  
  if len(text.split()) <30:
    text = the_norm_text(text, maxlen= 30)
  
  ls_text = text.split()[:30]
#   print(ls_text)
#   return len(ls_text)
  # vector visualize sequence information
  vec_sequence = []
  
  for word in ls_text:
    
    # tạo vector zeros (1,300)
    empyt_emb = np.zeros((1, 300))
    
    if word != '0':
      # vector visualize a word (1x300)
      vec_word = embeddings_index.get(word, empyt_emb)
    else:
      vec_word =  empyt_emb
    # reshape columns vector to row vector
    vec_word = vec_word.reshape((1, -1))
    
    vec_sequence += list(vec_word)
    
  # convert 2d array (30, 300) to 1d array (1, 9000)
  #return csr_matrix(np.reshape(np.array(vec_sequence), (1,np.product(np.array(vec_sequence).shape))))
  return np.reshape(np.array(vec_sequence), (1,np.product(np.array(vec_sequence).shape)))
  #return np.array(vec_sequence)

In [0]:
# Data providers
# số điểm dữ liệu trong 1 cụm
batch_size = 100

def batch_gen(train_df):
  # thực hiện chia tập dữ liệu thành n_batches cụm, mỗi cụm có batch_size phần tử 
  n_batches = math.ceil(len(train_df) / batch_size)
  while True: 
    # xáo trộn dữ liệu 
    train_df = train_df.sample(frac=1.)  # Shuffle the data.
     
    for i in range(n_batches):
      # thực hiện feature engineering cho cụm dữ liệu thứ i
      # tập dữ liệu của cta có các cột: pid, question_text, target, text_norm1, num_word_2
      # ta lấy list text ở côt text_norm , số lượng là batch_size = 128 điểm dữ liệu
      # vd: i=0 : train_df.iloc[i*batch_size:(i+1)*batch_size, 3] =>train_df.iloc[0*128:(0+1)*128, 3] 
      # vd: i=1 : train_df.iloc[i*batch_size:(i+1)*batch_size, 3] =>train_df.iloc[1*128:(1+1)*128, 3] 
      texts = train_df.iloc[i*batch_size:(i+1)*batch_size, 3]
      # với list text lấy được buowcs trên ta thực hiện convert text to vector cho từng dòng text và lưu vào 1 mảng
      for text in texts:
        assert text_to_vector(text).shape == (1, 9000), text
      text_arr = np.array([text_to_vector(text) for text in texts])
      
      # ta lấy nhãn tương ứng cho từng cụm dữ liệu khi trả về (y_train)
      # vd i= 0 : np.array(train_df["target"][i*batch_size:(i+1)*batch_size]) = np.array(train_df["target"][0*128:(0+1)*128])
      
      # yield là phương thức trả về nhiều lần của python
      # return trả về 1 lần
      #print(text_arr.shape)
      # reshape araay (128, 1, 9000) to (128, 9000)
      if i < n_batches - 1:
        text_arr =  text_arr.reshape((batch_size, -1))
      else:
        text_arr =  text_arr.reshape((text_arr.shape[0], -1))
      #assert text_arr.shape == (128, 9000)
      yield text_arr, np.array(train_df["target"][i*batch_size:(i+1)*batch_size])

**split train, validation set**

In [0]:
quora_train, quora_val = train_test_split(quora_train, test_size=0.06)

In [0]:
sentence_val = quora_val.text_norm1.values
y_val = quora_val.target.values
print(len(sentence_val))

In [0]:
val_vects = np.array([text_to_vector(x_val) for x_val in sentence_val])

In [0]:
val_vects.shape

In [0]:
val_vects.shape[2]

In [0]:
val_vects = val_vects.reshape(val_vects.shape[0], val_vects.shape[2])
val_vects.shape

# training

In [0]:
# # create neural network model
model = Sequential()
model.add(Dense(64, input_dim=9000, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
# reduce overfitting 
model.add(Dense(32, kernel_constraint=max_norm(3), bias_constraint=max_norm(3)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# setup optimizer para for model, use sgd
opt = SGD(lr=0.001,  decay=0.01)
model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

# bien save model
check_point = ModelCheckpoint('/content/drive/My Drive/quora-insincere-questions-classification/neural_network_emb_ver_01.h5', monitor='val_loss', save_best_only=True)

**steps_per_epoch**
+ Integer. Total number of steps (batches of samples) to yield from generator before declaring one epoch finished and starting the next epoch. It should typically be equal to ceil(num_samples / batch_size) Optional for Sequence: if unspecified, will use the len(generator) as a number of steps.

In [0]:
def input_noQuit(prompt):
    while True: #broken by return
        try:
            return input(prompt)
        except KeyboardInterrupt:
            print("you are not allowed to quit right now")

In [0]:
# Train the model, iterating on the data in batches of 32 samples
mg = batch_gen(quora_train)
input_noQuit(model.fit_generator(mg, epochs=20, steps_per_epoch=(math.ceil(len(quora_train) / batch_size)),validation_data=(val_vects, y_val),verbose=1, callbacks= [ check_point]))

In [0]:
all_preds = np.array([model.predict(text_to_vector(x)) for x in quora_test['text_norm1'] ])

In [0]:
a = all_preds.reshape(all_preds.shape[0],)

In [0]:
# choose with y_prod >= 0.29 => label = 1, y_prod < 0.29 => label = 0
y_class = np.squeeze(a >= 0.5).astype('int')

In [0]:
array_2 = y_class.reshape(1, -1)
print(array_2.shape)
array_2 = array_2[0]
print(array_2.shape)

In [0]:
print(classification_report(y_pred = array_2, y_true= y_test))